In [1]:
#Importing of relavent functions, pre-trained models, and libraries
import string
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation

def l1_distance(vectors):
    x, y = vectors
    return tf.reduce_sum(tf.abs(x - y), axis=1, keepdims=True)

import gensim.downloader
import numpy as np
from gensim.models import Word2Vec
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import cosine_similarity
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Lambda, Dense
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Embedder = gensim.downloader.load('word2vec-google-news-300')

In [2]:
#Reading in the training and test sets
Master = pd.read_csv("QQP/train.tsv",sep="\t")
Train,Dev = train_test_split(Master,test_size=0.1,random_state=42)
Test = pd.read_csv("QQP/dev.tsv",sep="\t")

In [3]:
#Pre-processing of training set

Train = Train.dropna()
Train = Train[~Train.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

for i in range(len(Train["question1"])):
    Train["question1"].iloc[i] = remove_punctuation(Train["question1"].iloc[i].lower())
    Train["question2"].iloc[i] = remove_punctuation(Train["question2"].iloc[i].lower())

Train = Train.dropna()
Train = Train[~Train.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

Q1 = Train["question1"].copy()
Q2 = Train["question2"].copy()

#Tokenization of training set
Q1_Tokens = Q1.copy()
Q2_Tokens = Q2.copy()
for i in range(len(Q1_Tokens)):
    Q1_Tokens.iloc[i] = Q1_Tokens.iloc[i].split()
    Q2_Tokens.iloc[i] = Q2_Tokens.iloc[i].split()

#Embedding of training set
Q1_Embedded = Q1_Tokens.copy()
Q2_Embedded = Q2_Tokens.copy()
for i in range(len(Q1_Embedded)):
  Sum_Q1 = np.zeros(300)
  Counter_Q1 = 0
  for word in Q1_Embedded.iloc[i]:
    if word in Embedder:
      Sum_Q1 = Sum_Q1 + Embedder[word]
      Counter_Q1 = Counter_Q1 + 1
  Q1_Embedded.iloc[i] = Sum_Q1 
  Sum_Q2 = np.zeros(300)
  Counter_Q2 = 0
  for word in Q2_Embedded.iloc[i]:
    if word in Embedder:
      Sum_Q2 = Sum_Q2 + Embedder[word]
      Counter_Q2 = Counter_Q2 + 1
  Q2_Embedded.iloc[i] = Sum_Q2


#Formatting inputs to match model's input format
Q1_Inputs = []
for i in range(len(Q1_Embedded)):
    #print(i)
    Q1_Inputs.append(Q1_Embedded.iloc[i].tolist())
Q1_Inputs = np.array(Q1_Inputs)

Q2_Inputs = []
for i in range(len(Q2_Embedded)):
    Q2_Inputs.append(Q2_Embedded.iloc[i].tolist())
Q2_Inputs = np.array(Q2_Inputs)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
#Pre-Processing of validation set
Dev = Dev.dropna()
Dev = Dev[~Dev.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

for i in range(len(Dev["question1"])):
    Dev["question1"].iloc[i] = remove_punctuation(Dev["question1"].iloc[i].lower())
    Dev["question2"].iloc[i] = remove_punctuation(Dev["question2"].iloc[i].lower())

Dev = Dev.dropna()
Dev = Dev[~Dev.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

Q1_Dev = Dev["question1"].copy()
Q2_Dev = Dev["question2"].copy()

#Tokenization of validation set
Q1_Dev_Tokens = Q1_Dev.copy()
Q2_Dev_Tokens = Q2_Dev.copy()
for i in range(len(Q1_Dev_Tokens)):
    Q1_Dev_Tokens.iloc[i] = Q1_Dev_Tokens.iloc[i].split()
    Q2_Dev_Tokens.iloc[i] = Q2_Dev_Tokens.iloc[i].split()

#Embedding of validation set
from gensim.models import Word2Vec
Q1_Embedded_Dev = Q1_Dev_Tokens.copy()
Q2_Embedded_Dev = Q2_Dev_Tokens.copy()

for i in range(len(Q1_Embedded_Dev)):
  Sum_Q1 = np.zeros(300)
  for word in Q1_Embedded_Dev.iloc[i]:
    if word in Embedder:
      Sum_Q1 = Sum_Q1 + (Embedder[word])
  Q1_Embedded_Dev.iloc[i] = Sum_Q1 
  Sum_Q2 = np.zeros(300)
  for word in Q2_Embedded_Dev.iloc[i]:
    if word in Embedder:
      Sum_Q2 = Sum_Q2 + (Embedder[word])
  Q2_Embedded_Dev.iloc[i] = Sum_Q2

#Formating embedding to match model's input format

Q1_Inputs_Dev = []
for i in range(len(Q1_Embedded_Dev)):
    Q1_Inputs_Dev.append(Q1_Embedded_Dev.iloc[i].tolist())
Q1_Inputs_Dev = np.array(Q1_Inputs_Dev)

Q2_Inputs_Dev = []
for i in range(len(Q2_Embedded_Dev)):
    Q2_Inputs_Dev.append(Q2_Embedded_Dev.iloc[i].tolist())
Q2_Inputs_Dev = np.array(Q2_Inputs_Dev)

In [5]:
#Pre-processing the test set
Test = Test.dropna()
Test = Test[~Test.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

for i in range(len(Test["question1"])):
    Test["question1"].iloc[i] = remove_punctuation(Test["question1"].iloc[i].lower())
    Test["question2"].iloc[i] = remove_punctuation(Test["question2"].iloc[i].lower())

Test = Test.dropna()
Test = Test[~Test.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

#Tokenization of the test set
Q1_Test = Test["question1"].copy()
Q2_Test = Test["question2"].copy()

Q1_Test_Tokens = Q1_Test.copy()
Q2_Test_Tokens = Q2_Test.copy()
for i in range(len(Q1_Test_Tokens)):
    Q1_Test_Tokens.iloc[i] = Q1_Test_Tokens.iloc[i].split()
    Q2_Test_Tokens.iloc[i] = Q2_Test_Tokens.iloc[i].split()

#Creating embeddings for the test set
from gensim.models import Word2Vec
Q1_Embedded_Test = Q1_Test_Tokens.copy()
Q2_Embedded_Test = Q2_Test_Tokens.copy()

for i in range(len(Q1_Embedded_Test)):
  Sum_Q1 = np.zeros(300)
  for word in Q1_Embedded_Test.iloc[i]:
    if word in Embedder:
      Sum_Q1 = Sum_Q1 + (Embedder[word])
  Q1_Embedded_Test.iloc[i] = Sum_Q1 
  Sum_Q2 = np.zeros(300)
  for word in Q2_Embedded_Test.iloc[i]:
    if word in Embedder:
      Sum_Q2 = Sum_Q2 + (Embedder[word])
  Q2_Embedded_Test.iloc[i] = Sum_Q2 

#Formatting the inputs to match the inputs of the model
Q1_Inputs_Test = []
for i in range(len(Q1_Embedded_Test)):
    Q1_Inputs_Test.append(Q1_Embedded_Test.iloc[i].tolist())
Q1_Inputs_Test = np.array(Q1_Inputs_Test)

Q2_Inputs_Test = []
for i in range(len(Q2_Embedded_Test)):
    Q2_Inputs_Test.append(Q2_Embedded_Test.iloc[i].tolist())
Q2_Inputs_Test = np.array(Q2_Inputs_Test)

In [6]:
#Clear up ram as we don't need the embedder anymore
Embedder = 0

In [7]:
#Initialization of the Siamese network with twin multi-layer perceptron models
embedding_dim = 300

shared_network = keras.Sequential([
    layers.Dense(300, activation='relu', input_shape=(embedding_dim,)),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
])

early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=5,           
    restore_best_weights=True  
)

left_input = layers.Input(shape=(embedding_dim,))
right_input = layers.Input(shape=(embedding_dim,))

encoded_left = shared_network(left_input)
encoded_right = shared_network(right_input)

distance = Lambda(l1_distance)([encoded_left, encoded_right])
similarity_prediction = Dense(1, activation='sigmoid')(distance)

siamese_model = keras.Model(inputs=[left_input, right_input], outputs=similarity_prediction)

custom_learning_rate = 0.0001
custom_optimizer = Adam(learning_rate=custom_learning_rate)

siamese_model.compile(loss='mean_squared_error', optimizer=custom_optimizer, metrics=['accuracy'])

Initialize the Siamese network architecture with the above model as the underlying architecture

Training of the model

In [8]:
#Training of the model

siamese_model.fit(
    [Q1_Inputs, Q2_Inputs],  # Your question embeddings
    np.array(Train["is_duplicate"]),  # Similarity labels (0 for dissimilar, 1 for similar)
    batch_size=64,
    epochs=25,
    validation_split=0.2  # You can adjust the validation split
    ,callbacks=[early_stopping]
)

Epoch 1/25
4094/4094 [==============================] - 26s 6ms/step - loss: 0.1880 - accuracy: 0.6966 - val_loss: 0.1760 - val_accuracy: 0.7347
Epoch 2/25
4094/4094 [==============================] - 24s 6ms/step - loss: 0.1664 - accuracy: 0.7551 - val_loss: 0.1653 - val_accuracy: 0.7632
Epoch 3/25
4094/4094 [==============================] - 25s 6ms/step - loss: 0.1515 - accuracy: 0.7839 - val_loss: 0.1556 - val_accuracy: 0.7772
Epoch 4/25
4094/4094 [==============================] - 25s 6ms/step - loss: 0.1390 - accuracy: 0.8066 - val_loss: 0.1488 - val_accuracy: 0.7871
Epoch 5/25
4094/4094 [==============================] - 25s 6ms/step - loss: 0.1276 - accuracy: 0.8264 - val_loss: 0.1438 - val_accuracy: 0.7958
Epoch 6/25
4094/4094 [==============================] - 25s 6ms/step - loss: 0.1171 - accuracy: 0.8452 - val_loss: 0.1438 - val_accuracy: 0.7964
Epoch 7/25
4094/4094 [==============================] - 27s 7ms/step - loss: 0.1070 - accuracy: 0.8621 - val_loss: 0.1411 - val_ac

In [9]:
#Predicting the training set
train_predictions = siamese_model.predict([Q1_Inputs, Q2_Inputs])
threshold = 0.5
train_binary_predictions = (train_predictions > threshold).astype(int)
training_accuracy = accuracy_score(Train["is_duplicate"].tolist(), train_binary_predictions)
print(f"Training Accuracy: {training_accuracy * 100:.2f}%")


10233/10233 [==============================] - 19s 2ms/step
Training Accuracy: 89.94%


In [10]:
#Predicting the validation set
Dev_predictions = siamese_model.predict([Q1_Inputs_Dev, Q2_Inputs_Dev])
threshold = 0.5
Dev_binary_predictions = (Dev_predictions > threshold).astype(int)
Dev_accuracy = accuracy_score(Dev["is_duplicate"].tolist(), Dev_binary_predictions)
print(f"Validation Accuracy: {Dev_accuracy * 100:.2f}%")

1137/1137 [==============================] - 2s 2ms/step
Validation Accuracy: 81.15%


In [11]:
#Predicting the test set
test_predictions = siamese_model.predict([Q1_Inputs_Test, Q2_Inputs_Test])
threshold = 0.5
test_binary_predictions = (test_predictions > threshold).astype(int)
test_accuracy = accuracy_score(Test["is_duplicate"].tolist(), test_binary_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


1264/1264 [==============================] - 2s 2ms/step
Test Accuracy: 80.68%
